<a href="https://colab.research.google.com/github/ChiroDeniro/ai-ml-projecten/blob/main/Learning_Recsys_Arithmetic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required packages (run this cell first in Colab)
!pip install gradio pandas numpy scikit-learn matplotlib seaborn -q

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import gradio as gr
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json

class ArithmeticRecommender:
    """
    Adaptive Arithmetic Learning Recommender System

    Combines:
    - Content-based filtering (skill similarity)
    - Collaborative filtering (learner similarity)
    - Performance-based adaptive difficulty
    """

    def __init__(self):
        # Define problem types with metadata
        self.problem_types = {
            'addition_1digit': {'difficulty': 1, 'prereq': [], 'skills': ['counting', 'number_sense']},
            'addition_2digit': {'difficulty': 2, 'prereq': ['addition_1digit'], 'skills': ['counting', 'place_value']},
            'addition_3digit': {'difficulty': 3, 'prereq': ['addition_2digit'], 'skills': ['place_value', 'carrying']},
            'subtraction_1digit': {'difficulty': 1, 'prereq': [], 'skills': ['counting', 'number_sense']},
            'subtraction_2digit': {'difficulty': 2, 'prereq': ['subtraction_1digit'], 'skills': ['place_value', 'borrowing']},
            'subtraction_3digit': {'difficulty': 3, 'prereq': ['subtraction_2digit'], 'skills': ['place_value', 'borrowing']},
            'multiplication_1digit': {'difficulty': 2, 'prereq': ['addition_2digit'], 'skills': ['skip_counting', 'memorization']},
            'multiplication_2digit': {'difficulty': 4, 'prereq': ['multiplication_1digit'], 'skills': ['place_value', 'carrying']},
            'division_1digit': {'difficulty': 3, 'prereq': ['multiplication_1digit'], 'skills': ['inverse_operations', 'memorization']},
            'division_2digit': {'difficulty': 5, 'prereq': ['division_1digit', 'multiplication_2digit'], 'skills': ['long_division', 'estimation']},
            'fractions_basic': {'difficulty': 4, 'prereq': ['division_1digit'], 'skills': ['part_whole', 'equivalence']},
            'fractions_operations': {'difficulty': 5, 'prereq': ['fractions_basic'], 'skills': ['common_denominator', 'simplification']},
            'decimals_basic': {'difficulty': 3, 'prereq': ['division_1digit'], 'skills': ['place_value', 'decimal_notation']},
            'decimals_operations': {'difficulty': 4, 'prereq': ['decimals_basic'], 'skills': ['decimal_arithmetic', 'precision']},
            'percentages': {'difficulty': 4, 'prereq': ['fractions_basic', 'decimals_basic'], 'skills': ['proportional_reasoning', 'conversion']}
        }

        # Initialize learner data storage
        self.learner_history = {}
        self.learner_profiles = {}

        # Create problem feature matrix for content-based filtering
        self.create_problem_features()

    def create_problem_features(self):
        """Create feature matrix for problems (content-based filtering)"""
        problems = list(self.problem_types.keys())

        # Extract all unique skills
        all_skills = set()
        for prob in self.problem_types.values():
            all_skills.update(prob['skills'])
        all_skills = sorted(list(all_skills))

        # Create feature matrix: [difficulty, skill1, skill2, ...]
        features = []
        for prob_name in problems:
            prob = self.problem_types[prob_name]
            row = [prob['difficulty']]
            for skill in all_skills:
                row.append(1 if skill in prob['skills'] else 0)
            features.append(row)

        self.problem_features = np.array(features)
        self.problem_names = problems
        self.skill_names = all_skills

        # Normalize features
        scaler = StandardScaler()
        self.problem_features_normalized = scaler.fit_transform(self.problem_features)

    def calculate_performance_score(self, learner_id, problem_type):
        """Calculate learner's performance on a specific problem type"""
        if learner_id not in self.learner_history:
            return 0.5  # Neutral starting point

        history = self.learner_history[learner_id]
        attempts = [h for h in history if h['problem_type'] == problem_type]

        if not attempts:
            return 0.5

        # Weighted average favoring recent attempts
        weights = np.exp(np.linspace(-1, 0, len(attempts)))
        scores = np.array([a['correct'] for a in attempts])
        return np.average(scores, weights=weights)

    def get_learner_profile(self, learner_id):
        """Generate learner skill profile vector"""
        if learner_id not in self.learner_history:
            return np.ones(len(self.problem_names)) * 0.5

        profile = []
        for prob_name in self.problem_names:
            profile.append(self.calculate_performance_score(learner_id, prob_name))

        return np.array(profile)

    def collaborative_filtering_score(self, learner_id, problem_type):
        """Find similar learners and predict performance"""
        if len(self.learner_history) < 2:
            return 0.5

        # Build learner-problem matrix
        learners = list(self.learner_history.keys())
        if learner_id not in learners:
            return 0.5

        matrix = []
        for lid in learners:
            matrix.append(self.get_learner_profile(lid))

        matrix = np.array(matrix)

        # Calculate learner similarity
        learner_idx = learners.index(learner_id)
        similarities = cosine_similarity([matrix[learner_idx]], matrix)[0]

        # Weight other learners' performance by similarity
        problem_idx = self.problem_names.index(problem_type)
        weighted_scores = []

        for i, other_learner in enumerate(learners):
            if i != learner_idx and similarities[i] > 0:
                weighted_scores.append(similarities[i] * matrix[i][problem_idx])

        if not weighted_scores:
            return 0.5

        return np.mean(weighted_scores)

    def content_based_score(self, learner_id, problem_type):
        """Recommend based on similar problems the learner did well on"""
        profile = self.get_learner_profile(learner_id)

        # Find problems learner performed well on
        strong_problems = [i for i, score in enumerate(profile) if score > 0.7]

        if not strong_problems:
            # If no strong areas, recommend easier problems
            problem_idx = self.problem_names.index(problem_type)
            return 1.0 / (self.problem_features[problem_idx][0] + 1)

        # Calculate similarity to strong problem areas
        problem_idx = self.problem_names.index(problem_type)
        similarities = cosine_similarity(
            [self.problem_features_normalized[problem_idx]],
            self.problem_features_normalized[strong_problems]
        )[0]

        return np.mean(similarities)

    def check_prerequisites(self, learner_id, problem_type):
        """Check if learner has mastered prerequisites"""
        prereqs = self.problem_types[problem_type]['prereq']

        if not prereqs:
            return 1.0

        prereq_scores = [self.calculate_performance_score(learner_id, p) for p in prereqs]

        if not prereq_scores:
            return 0.5

        return np.mean(prereq_scores)

    def recommend_next_problems(self, learner_id, n=5):
        """
        Hybrid recommendation combining multiple signals
        """
        recommendations = []

        for problem_type in self.problem_names:
            # Current performance
            current_score = self.calculate_performance_score(learner_id, problem_type)

            # Collaborative filtering
            cf_score = self.collaborative_filtering_score(learner_id, problem_type)

            # Content-based filtering
            cb_score = self.content_based_score(learner_id, problem_type)

            # Prerequisites check
            prereq_score = self.check_prerequisites(learner_id, problem_type)

            # Difficulty appropriateness (challenge but not too hard)
            difficulty = self.problem_types[problem_type]['difficulty']
            learner_level = np.mean(self.get_learner_profile(learner_id)) * 5
            difficulty_score = 1.0 - abs(difficulty - learner_level - 0.5) / 5.0
            difficulty_score = max(0, difficulty_score)

            # Combine scores (weighted ensemble)
            # Prioritize problems learner hasn't mastered but is ready for
            mastery_gap = max(0, 0.8 - current_score)  # Want to improve weak areas

            final_score = (
                0.15 * cf_score +
                0.15 * cb_score +
                0.30 * prereq_score +
                0.20 * difficulty_score +
                0.20 * mastery_gap
            )

            recommendations.append({
                'problem_type': problem_type,
                'score': final_score,
                'current_performance': current_score,
                'difficulty': difficulty,
                'details': {
                    'collaborative': cf_score,
                    'content_based': cb_score,
                    'prerequisites': prereq_score,
                    'difficulty_match': difficulty_score,
                    'growth_potential': mastery_gap
                }
            })

        # Sort by score and return top N
        recommendations.sort(key=lambda x: x['score'], reverse=True)
        return recommendations[:n]

    def record_attempt(self, learner_id, problem_type, correct, time_spent=None):
        """Record a learning attempt"""
        if learner_id not in self.learner_history:
            self.learner_history[learner_id] = []

        self.learner_history[learner_id].append({
            'problem_type': problem_type,
            'correct': 1 if correct else 0,
            'time_spent': time_spent,
            'timestamp': datetime.now().isoformat()
        })

    def get_learner_stats(self, learner_id):
        """Generate learner statistics"""
        if learner_id not in self.learner_history:
            return "No history yet for this learner."

        history = self.learner_history[learner_id]

        total_attempts = len(history)
        correct_attempts = sum(h['correct'] for h in history)
        accuracy = correct_attempts / total_attempts if total_attempts > 0 else 0

        # Performance by problem type
        problem_stats = {}
        for prob_name in self.problem_names:
            score = self.calculate_performance_score(learner_id, prob_name)
            attempts = len([h for h in history if h['problem_type'] == prob_name])
            problem_stats[prob_name] = {
                'score': score,
                'attempts': attempts
            }

        return {
            'total_attempts': total_attempts,
            'accuracy': accuracy,
            'problem_stats': problem_stats
        }

# Initialize the recommender system
recommender = ArithmeticRecommender()

# Simulate some learner data for demonstration
def populate_sample_data():
    """Create sample learner histories"""
    learners = ['Alice', 'Bob', 'Charlie', 'Diana']

    # Alice: Strong in basics, working on advanced
    for _ in range(10):
        recommender.record_attempt('Alice', 'addition_1digit', True)
        recommender.record_attempt('Alice', 'addition_2digit', True)
        recommender.record_attempt('Alice', 'subtraction_1digit', True)
    for _ in range(5):
        recommender.record_attempt('Alice', 'multiplication_1digit', np.random.random() > 0.3)
        recommender.record_attempt('Alice', 'division_1digit', np.random.random() > 0.5)

    # Bob: Struggling with basics
    for _ in range(15):
        recommender.record_attempt('Bob', 'addition_1digit', np.random.random() > 0.4)
        recommender.record_attempt('Bob', 'subtraction_1digit', np.random.random() > 0.5)
        recommender.record_attempt('Bob', 'addition_2digit', np.random.random() > 0.7)

    # Charlie: Advanced learner
    for prob in ['addition_2digit', 'subtraction_2digit', 'multiplication_1digit', 'division_1digit']:
        for _ in range(8):
            recommender.record_attempt('Charlie', prob, True)
    for _ in range(5):
        recommender.record_attempt('Charlie', 'fractions_basic', np.random.random() > 0.3)
        recommender.record_attempt('Charlie', 'decimals_basic', np.random.random() > 0.4)

    # Diana: Well-rounded, working on fractions
    for prob in ['addition_2digit', 'subtraction_2digit', 'multiplication_1digit']:
        for _ in range(6):
            recommender.record_attempt('Diana', prob, True)
    for _ in range(4):
        recommender.record_attempt('Diana', 'fractions_basic', np.random.random() > 0.4)

populate_sample_data()

# Gradio Interface Functions
def get_recommendations(learner_name, num_recommendations):
    """Get recommendations for a learner"""
    recommendations = recommender.recommend_next_problems(learner_name, n=num_recommendations)

    output = f"### 📚 Recommended Problems for {learner_name}\n\n"

    for i, rec in enumerate(recommendations, 1):
        output += f"**{i}. {rec['problem_type'].replace('_', ' ').title()}**\n"
        output += f"   - Recommendation Score: {rec['score']:.2f}\n"
        output += f"   - Current Performance: {rec['current_performance']:.1%}\n"
        output += f"   - Difficulty Level: {rec['difficulty']}/5\n"
        output += f"   - Growth Potential: {rec['details']['growth_potential']:.2f}\n\n"

    return output

def record_practice(learner_name, problem_type, correct):
    """Record a practice attempt"""
    problem_type = problem_type.lower().replace(' ', '_')

    if problem_type not in recommender.problem_names:
        return f"❌ Problem type '{problem_type}' not found. Please check spelling."

    recommender.record_attempt(learner_name, problem_type, correct == "Correct ✓")

    return f"✅ Recorded: {learner_name} attempted {problem_type} - {'Correct' if correct == 'Correct ✓' else 'Incorrect'}"

def show_learner_profile(learner_name):
    """Display learner's skill profile"""
    stats = recommender.get_learner_stats(learner_name)

    if isinstance(stats, str):
        return stats

    output = f"### 📊 Learning Profile: {learner_name}\n\n"
    output += f"**Overall Stats:**\n"
    output += f"- Total Attempts: {stats['total_attempts']}\n"
    output += f"- Overall Accuracy: {stats['accuracy']:.1%}\n\n"

    output += f"**Performance by Topic:**\n\n"

    # Sort by performance
    sorted_problems = sorted(
        stats['problem_stats'].items(),
        key=lambda x: x[1]['score'],
        reverse=True
    )

    for prob_name, prob_stats in sorted_problems:
        if prob_stats['attempts'] > 0:
            output += f"- {prob_name.replace('_', ' ').title()}: "
            output += f"{prob_stats['score']:.1%} ({prob_stats['attempts']} attempts)\n"

    return output

def create_visualization(learner_name):
    """Create performance visualization"""
    profile = recommender.get_learner_profile(learner_name)

    if len(recommender.learner_history.get(learner_name, [])) == 0:
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.text(0.5, 0.5, 'No data available',
                ha='center', va='center', fontsize=16)
        ax.axis('off')
        return fig

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

    # Skill heatmap
    skills_performance = []
    skill_labels = []

    for prob_name in recommender.problem_names:
        score = recommender.calculate_performance_score(learner_name, prob_name)
        attempts = len([h for h in recommender.learner_history[learner_name]
                       if h['problem_type'] == prob_name])
        if attempts > 0:
            skills_performance.append(score)
            skill_labels.append(prob_name.replace('_', '\n'))

    if skills_performance:
        colors = ['#d32f2f' if s < 0.5 else '#ffa726' if s < 0.7 else '#66bb6a'
                 for s in skills_performance]
        ax1.barh(range(len(skill_labels)), skills_performance, color=colors)
        ax1.set_yticks(range(len(skill_labels)))
        ax1.set_yticklabels(skill_labels, fontsize=8)
        ax1.set_xlabel('Performance Score')
        ax1.set_title(f'{learner_name}: Skill Mastery')
        ax1.set_xlim([0, 1])
        ax1.axvline(x=0.7, color='gray', linestyle='--', alpha=0.5)

    # Learning curve
    history = recommender.learner_history[learner_name]
    cumulative_accuracy = []
    for i in range(1, len(history) + 1):
        acc = sum(h['correct'] for h in history[:i]) / i
        cumulative_accuracy.append(acc)

    ax2.plot(cumulative_accuracy, linewidth=2, color='#2196f3')
    ax2.fill_between(range(len(cumulative_accuracy)), cumulative_accuracy,
                      alpha=0.3, color='#2196f3')
    ax2.set_xlabel('Attempts')
    ax2.set_ylabel('Cumulative Accuracy')
    ax2.set_title(f'{learner_name}: Learning Progress')
    ax2.set_ylim([0, 1])
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    return fig

# Create Gradio Interface
with gr.Blocks(title="Adaptive Arithmetic Learning Recommender", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🎓 Adaptive Arithmetic Learning Recommender System

    **A demonstration of hybrid recommendation techniques for personalized learning**

    This system combines:
    - 🤝 **Collaborative Filtering**: Learn from similar students
    - 📖 **Content-Based Filtering**: Match problems by skills and difficulty
    - 📈 **Adaptive Difficulty**: Challenge students at the right level
    - ✅ **Prerequisite Tracking**: Ensure foundational knowledge

    Sample learners: Alice, Bob, Charlie, Diana (or create your own!)
    """)

    with gr.Tab("🎯 Get Recommendations"):
        with gr.Row():
            with gr.Column():
                learner_input = gr.Textbox(
                    label="Learner Name",
                    placeholder="Enter name (e.g., Alice, Bob, or your name)",
                    value="Alice"
                )
                num_recs = gr.Slider(
                    minimum=1,
                    maximum=10,
                    value=5,
                    step=1,
                    label="Number of Recommendations"
                )
                rec_button = gr.Button("Get Recommendations", variant="primary")

            with gr.Column():
                rec_output = gr.Markdown()

        rec_button.click(
            fn=get_recommendations,
            inputs=[learner_input, num_recs],
            outputs=rec_output
        )

    with gr.Tab("✏️ Record Practice"):
        with gr.Row():
            with gr.Column():
                practice_learner = gr.Textbox(label="Learner Name", value="Alice")
                practice_problem = gr.Dropdown(
                    choices=[p.replace('_', ' ').title() for p in recommender.problem_names],
                    label="Problem Type",
                    value="Addition 1digit"
                )
                practice_correct = gr.Radio(
                    choices=["Correct ✓", "Incorrect ✗"],
                    label="Result",
                    value="Correct ✓"
                )
                practice_button = gr.Button("Record Attempt", variant="primary")

            with gr.Column():
                practice_output = gr.Textbox(label="Status")

        practice_button.click(
            fn=record_practice,
            inputs=[practice_learner, practice_problem, practice_correct],
            outputs=practice_output
        )

    with gr.Tab("📊 Learner Profile"):
        with gr.Row():
            with gr.Column(scale=1):
                profile_learner = gr.Textbox(label="Learner Name", value="Alice")
                profile_button = gr.Button("View Profile", variant="primary")
                profile_output = gr.Markdown()

            with gr.Column(scale=2):
                viz_output = gr.Plot()

        profile_button.click(
            fn=show_learner_profile,
            inputs=[profile_learner],
            outputs=profile_output
        )

        profile_button.click(
            fn=create_visualization,
            inputs=[profile_learner],
            outputs=viz_output
        )

    with gr.Tab("ℹ️ System Info"):
        gr.Markdown("""
        ## How It Works

        ### 1. Collaborative Filtering
        Finds learners with similar performance patterns and recommends problems they succeeded with.

        ### 2. Content-Based Filtering
        Recommends problems similar to ones you've already mastered, based on:
        - Required skills (place value, carrying, etc.)
        - Difficulty level
        - Mathematical concepts

        ### 3. Adaptive Difficulty
        Matches problems to your current level:
        - Not too easy (already mastered)
        - Not too hard (missing prerequisites)
        - Just right (optimal challenge)

        ### 4. Performance Tracking
        - Tracks mastery level for each topic (0-100%)
        - Recent attempts weighted more heavily
        - Identifies growth areas

        ## Problem Types
        """)

        problem_info = "| Problem Type | Difficulty | Prerequisites | Skills |\n"
        problem_info += "|--------------|------------|---------------|--------|\n"

        for name, details in recommender.problem_types.items():
            prereqs = ', '.join(details['prereq']) if details['prereq'] else 'None'
            skills = ', '.join(details['skills'])
            problem_info += f"| {name.replace('_', ' ').title()} | {details['difficulty']}/5 | {prereqs} | {skills} |\n"

        gr.Markdown(problem_info)

# Launch the interface
demo.launch(share=True)

/tmp/ipython-input-929324459.py:415: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="Adaptive Arithmetic Learning Recommender", theme=gr.themes.Soft()) as demo:
/usr/local/lib/python3.12/dist-packages/gradio/components/dropdown.py:230: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include: Addition 1digit or set allow_custom_value=True.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://403a20531d9651304e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
